## Mathematical Foundation

### The SES Formula

$$\hat{y}_{t+1|t} = \alpha y_t + (1-\alpha) \hat{y}_{t|t-1}$$

Where:
- $\hat{y}_{t+1|t}$ = forecast for time $t+1$ made at time $t$
- $\alpha$ = smoothing parameter (0 < α < 1)
- $y_t$ = actual observation at time $t$
- $\hat{y}_{t|t-1}$ = forecast made at time $t-1$ for time $t$

### Alternative Form (Weighted Average)

$$\hat{y}_{t+1|t} = \sum_{i=0}^{t-1} \alpha (1-\alpha)^i y_{t-i}$$

This shows that SES is a weighted average of all past observations with exponentially decaying weights.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Load and Prepare Data

We'll use gold price data, specifically the differenced series which is stationary (as confirmed in Day 6).

In [2]:
# Load gold price data
gold = pd.read_csv("../data/gold_prices.csv", parse_dates=["Date"], index_col="Date")
gold['Price'] = (gold['Price'].astype(float) * 10.8).round(0)

# Create differenced series (stationary)
gold['Price_Diff'] = gold['Price'].diff()
gold_diff = gold['Price_Diff'].dropna()

print(f"Original data shape: {gold.shape}")
print(f"Differenced data shape: {gold_diff.shape}")
print(f"\nDate range: {gold_diff.index.min()} to {gold_diff.index.max()}")
print(f"\nDifferenced series statistics:")
print(gold_diff.describe())

Original data shape: (2515, 2)
Differenced data shape: (2514,)

Date range: 2016-01-12 00:00:00 to 2026-01-09 00:00:00

Differenced series statistics:
count    2514.000000
mean        1.329753
std        21.016656
min      -280.000000
25%        -7.000000
50%         1.000000
75%         9.000000
max       153.000000
Name: Price_Diff, dtype: float64


## Train-Test Split

We'll use 80% for training and 20% for testing to evaluate forecast performance.

In [3]:
# Split data: 80% train, 20% test
train_size = int(len(gold_diff) * 0.8)
train = gold_diff.iloc[:train_size]
test = gold_diff.iloc[train_size:]

print(f"Training set: {len(train)} observations ({train.index.min()} to {train.index.max()})")
print(f"Test set: {len(test)} observations ({test.index.min()} to {test.index.max()})")
print(f"\nTrain/Test split: {len(train)/len(gold_diff)*100:.1f}% / {len(test)/len(gold_diff)*100:.1f}%")

Training set: 2011 observations (2016-01-12 00:00:00 to 2024-01-08 00:00:00)
Test set: 503 observations (2024-01-09 00:00:00 to 2026-01-09 00:00:00)

Train/Test split: 80.0% / 20.0%


## Understanding the Alpha Parameter

The smoothing parameter α controls the weight given to recent vs. historical observations:

- **α close to 1**: More weight on recent observations (responsive, less smooth)
- **α close to 0**: More weight on historical observations (smooth, less responsive)

### Weight Decay Example

For α = 0.3:
- Current observation: 30%
- 1 period ago: 30% × 0.7 = 21%
- 2 periods ago: 30% × 0.7² = 14.7%
- 3 periods ago: 30% × 0.7³ = 10.3%

## Manual SES Implementation

Let's implement SES from scratch to understand the mechanics.

In [4]:
def manual_ses(data, alpha):
    """
    Manual Simple Exponential Smoothing implementation.
    
    Parameters:
    -----------
    data : array-like
        Time series data
    alpha : float
        Smoothing parameter (0 < alpha < 1)
    
    Returns:
    --------
    fitted : array
        Fitted values (one-step ahead forecasts)
    """
    fitted = np.zeros(len(data))
    fitted[0] = data[0]  # Initialize with first observation
    
    for t in range(1, len(data)):
        fitted[t] = alpha * data[t-1] + (1 - alpha) * fitted[t-1]
    
    return fitted

# Test manual implementation
alpha_test = 0.3
manual_fit = manual_ses(train.values, alpha_test)

print(f"Manual SES with α = {alpha_test}")
print(f"First 5 fitted values: {manual_fit[:5]}")
print(f"Last 5 fitted values: {manual_fit[-5:]}")

Manual SES with α = 0.3
First 5 fitted values: [-6.    -6.    -2.4   -7.08  -1.656]
Last 5 fitted values: [ 1.05424883 -0.76202582 -5.63341807 -3.34339265 -2.34037486]


## SES with Statsmodels

Now let's use the statsmodels implementation, which includes optimization for finding the best α.

In [5]:
# Fit SES model with optimized alpha
model = SimpleExpSmoothing(train)
fitted_model = model.fit(optimized=True)

# Get optimal alpha
optimal_alpha = fitted_model.params['smoothing_level']

print(f"\nOptimized α (smoothing level): {optimal_alpha:.4f}")
print(f"\nModel Summary:")
print(fitted_model.summary())


Optimized α (smoothing level): 0.0085

Model Summary:
                       SimpleExpSmoothing Model Results                       
Dep. Variable:             Price_Diff   No. Observations:                 2011
Model:             SimpleExpSmoothing   SSE                         436935.262
Optimized:                       True   AIC                          10825.499
Trend:                           None   BIC                          10836.711
Seasonal:                        None   AICC                         10825.518
Seasonal Periods:                None   Date:                 Fri, 16 Jan 2026
Box-Cox:                        False   Time:                         13:37:15
Box-Cox Coeff.:                  None                                         
                       coeff                 code              optimized      
------------------------------------------------------------------------------
smoothing_level            0.0085298                alpha                 Tr

## Alpha Parameter Tuning

Let's test different α values to understand their impact on forecast accuracy.

In [6]:
# Test range of alpha values
alphas = np.arange(0.1, 1.0, 0.1)
results = []

for alpha in alphas:
    # Fit model with specific alpha
    model = SimpleExpSmoothing(train)
    fit = model.fit(smoothing_level=alpha, optimized=False)
    
    # In-sample predictions
    fitted_values = fit.fittedvalues
    
    # Calculate in-sample errors
    mse = mean_squared_error(train[1:], fitted_values[1:])
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(train[1:], fitted_values[1:])
    
    results.append({
        'alpha': alpha,
        'mse': mse,
        'rmse': rmse,
        'mae': mae
    })

results_df = pd.DataFrame(results)

print("\nAlpha Parameter Tuning Results:")
print("=" * 60)
print(results_df.to_string(index=False))
print("\nBest alpha by RMSE:", results_df.loc[results_df['rmse'].idxmin(), 'alpha'])


Alpha Parameter Tuning Results:
 alpha        mse      rmse       mae
   0.1 227.350029 15.078131 10.666274
   0.2 239.721995 15.482958 11.058006
   0.3 253.314771 15.915865 11.463926
   0.4 268.459633 16.384738 11.882881
   0.5 285.521321 16.897376 12.335313
   0.6 304.919189 17.461935 12.819973
   0.7 327.142521 18.087082 13.339595
   0.8 352.815510 18.783384 13.901440
   0.9 382.794471 19.565134 14.508581

Best alpha by RMSE: 0.1


## Visualize Alpha Effects

In [7]:
# Create visualization of alpha effects
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('α = 0.1 (Very Smooth)', 'α = 0.3 (Balanced)', 
                    'α = 0.7 (Responsive)', 'α = 0.9 (Very Responsive)')
)

test_alphas = [0.1, 0.3, 0.7, 0.9]
positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

# Use last 200 points for clarity
train_viz = train.iloc[-200:]

for alpha, (row, col) in zip(test_alphas, positions):
    model = SimpleExpSmoothing(train_viz)
    fit = model.fit(smoothing_level=alpha, optimized=False)
    
    # Actual data
    fig.add_trace(
        go.Scatter(
            x=train_viz.index,
            y=train_viz.values,
            mode='lines',
            name='Actual',
            line=dict(color='gray', width=1),
            opacity=0.5,
            showlegend=(row==1 and col==1)
        ),
        row=row, col=col
    )
    
    # Fitted values
    fig.add_trace(
        go.Scatter(
            x=train_viz.index,
            y=fit.fittedvalues,
            mode='lines',
            name=f'SES (α={alpha})',
            line=dict(color='#FFD700', width=2),
            showlegend=(row==1 and col==1)
        ),
        row=row, col=col
    )

fig.update_layout(
    title='<b>Effect of Alpha Parameter on SES Smoothing</b>',
    height=800,
    template='plotly_dark',
    showlegend=True
)

fig.show()

## Generate Forecasts

Now let's generate out-of-sample forecasts using the optimized model.

In [8]:
# Fit model on full training data with optimized alpha
model_final = SimpleExpSmoothing(train)
fit_final = model_final.fit(optimized=True)

# Generate forecasts for test period
forecast_steps = len(test)
forecast = fit_final.forecast(steps=forecast_steps)

print(f"\nForecasting with α = {fit_final.params['smoothing_level']:.4f}")
print(f"Forecast horizon: {forecast_steps} steps")
print(f"\nFirst 10 forecasts:")
print(forecast.head(10))
print(f"\nLast 10 forecasts:")
print(forecast.tail(10))


Forecasting with α = 0.0085
Forecast horizon: 503 steps

First 10 forecasts:
2011    0.717524
2012    0.717524
2013    0.717524
2014    0.717524
2015    0.717524
2016    0.717524
2017    0.717524
2018    0.717524
2019    0.717524
2020    0.717524
dtype: float64

Last 10 forecasts:
2504    0.717524
2505    0.717524
2506    0.717524
2507    0.717524
2508    0.717524
2509    0.717524
2510    0.717524
2511    0.717524
2512    0.717524
2513    0.717524
dtype: float64


## Evaluate Forecast Performance

In [9]:
# Calculate forecast errors
forecast_mse = mean_squared_error(test, forecast)
forecast_rmse = np.sqrt(forecast_mse)
forecast_mae = mean_absolute_error(test, forecast)

# Calculate Mean Absolute Percentage Error
mape = np.mean(np.abs((test - forecast) / test)) * 100

print("\n" + "=" * 60)
print("FORECAST PERFORMANCE METRICS")
print("=" * 60)
print(f"Mean Squared Error (MSE): {forecast_mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {forecast_rmse:.4f}")
print(f"Mean Absolute Error (MAE): {forecast_mae:.4f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Compare to naive forecast (using last observation)
naive_forecast = np.full(len(test), train.iloc[-1])
naive_rmse = np.sqrt(mean_squared_error(test, naive_forecast))

print(f"\nBaseline (Naive) RMSE: {naive_rmse:.4f}")
print(f"Improvement over naive: {((naive_rmse - forecast_rmse) / naive_rmse * 100):.2f}%")


FORECAST PERFORMANCE METRICS
Mean Squared Error (MSE): 1350.9051
Root Mean Squared Error (RMSE): 36.7547
Mean Absolute Error (MAE): 25.3474
Mean Absolute Percentage Error (MAPE): nan%

Baseline (Naive) RMSE: 42.0584
Improvement over naive: 12.61%


## Visualize Forecasts vs Actuals

In [16]:
# Create comprehensive forecast visualization
fig = go.Figure()

# Training data
fig.add_trace(go.Scatter(
    x=train.index,
    y=train.values,
    mode='lines',
    name='Training Data',
    line=dict(color='#4ECDC4', width=1.5),
    hovertemplate='<b>Date:</b> %{x}<br><b>Value:</b> %{y:.2f}<extra></extra>'
))

# Test data (actual)
fig.add_trace(go.Scatter(
    x=test.index,
    y=test.values,
    mode='lines',
    name='Actual Test Data',
    line=dict(color='#FF6B6B', width=2),
    hovertemplate='<b>Date:</b> %{x}<br><b>Actual:</b> %{y:.2f}<extra></extra>'
))

# Forecasts
fig.add_trace(go.Scatter(
    x=test.index,
    y=forecast.values,
    mode='lines',
    name=f'SES Forecast (α={fit_final.params["smoothing_level"]:.3f})',
    line=dict(color='#FFD700', width=2, dash='dash'),
    hovertemplate='<b>Date:</b> %{x}<br><b>Forecast:</b> %{y:.2f}<extra></extra>'
))

# Add vertical line at train/test split
fig.add_vline(
    x=test.index[0],
    line_dash="dot",
    line_color="white",
    annotation_text="Forecast Start",
    annotation_position="top"
)

fig.update_layout(
    title='<b>Simple Exponential Smoothing: Training, Test, and Forecast</b>',
    xaxis_title='Date',
    yaxis_title='Price Change (Differenced)',
    hovermode='x unified',
    height=600,
    template='plotly_dark',
    legend=dict(x=0.01, y=0.99)
)

fig.show()

TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

## Residual Analysis

Analyze the forecast errors to assess model quality.

In [11]:
# Calculate residuals
residuals = test - forecast

print("\n" + "=" * 60)
print("RESIDUAL ANALYSIS")
print("=" * 60)
print(f"\nResidual Statistics:")
print(residuals.describe())
print(f"\nResidual Mean (should be ~0): {residuals.mean():.4f}")
print(f"Residual Std Dev: {residuals.std():.4f}")


RESIDUAL ANALYSIS

Residual Statistics:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
dtype: float64

Residual Mean (should be ~0): nan
Residual Std Dev: nan


In [12]:
# Visualize residuals
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Residuals Over Time', 'Residual Distribution')
)

# Residuals time series
fig.add_trace(
    go.Scatter(
        x=residuals.index,
        y=residuals.values,
        mode='lines+markers',
        name='Residuals',
        line=dict(color='#FF6B6B', width=1),
        marker=dict(size=3)
    ),
    row=1, col=1
)

# Add zero line
fig.add_hline(
    y=0,
    line_dash="dash",
    line_color="white",
    row=1, col=1
)

# Residual histogram
fig.add_trace(
    go.Histogram(
        x=residuals.values,
        nbinsx=30,
        name='Distribution',
        marker=dict(color='#FFD700')
    ),
    row=1, col=2
)

fig.update_layout(
    title='<b>Residual Analysis</b>',
    height=400,
    template='plotly_dark',
    showlegend=False
)

fig.update_xaxes(title_text='Date', row=1, col=1)
fig.update_xaxes(title_text='Residual Value', row=1, col=2)
fig.update_yaxes(title_text='Residual', row=1, col=1)
fig.update_yaxes(title_text='Frequency', row=1, col=2)

fig.show()

## Compare Multiple Alpha Values

In [13]:
# Generate forecasts with different alphas
comparison_alphas = [0.2, 0.5, 0.8, optimal_alpha]
forecast_comparison = {}

for alpha in comparison_alphas:
    model = SimpleExpSmoothing(train)
    if alpha == optimal_alpha:
        fit = model.fit(optimized=True)
        label = f'α={alpha:.3f} (optimal)'
    else:
        fit = model.fit(smoothing_level=alpha, optimized=False)
        label = f'α={alpha:.1f}'
    
    fc = fit.forecast(steps=len(test))
    rmse = np.sqrt(mean_squared_error(test, fc))
    forecast_comparison[label] = {'forecast': fc, 'rmse': rmse}

print("\n" + "=" * 60)
print("ALPHA COMPARISON ON TEST SET")
print("=" * 60)
for label, data in forecast_comparison.items():
    print(f"{label}: RMSE = {data['rmse']:.4f}")


ALPHA COMPARISON ON TEST SET
α=0.2: RMSE = 37.6601
α=0.5: RMSE = 39.0721
α=0.8: RMSE = 40.5894
α=0.009 (optimal): RMSE = 36.7547


In [14]:
# Visualize comparison
fig = go.Figure()

# Actual test data
fig.add_trace(go.Scatter(
    x=test.index,
    y=test.values,
    mode='lines',
    name='Actual',
    line=dict(color='white', width=2)
))

# Different forecasts
colors = ['#4ECDC4', '#FFD700', '#FF6B6B', '#95E1D3']
for (label, data), color in zip(forecast_comparison.items(), colors):
    fig.add_trace(go.Scatter(
        x=test.index,
        y=data['forecast'].values,
        mode='lines',
        name=f'{label} (RMSE={data["rmse"]:.2f})',
        line=dict(color=color, width=2, dash='dash')
    ))

fig.update_layout(
    title='<b>Forecast Comparison: Different Alpha Values</b>',
    xaxis_title='Date',
    yaxis_title='Price Change',
    hovermode='x unified',
    height=600,
    template='plotly_dark',
    legend=dict(x=0.01, y=0.99)
)

fig.show()

## Key Insights and Recommendations

### When to Use SES:
- ✅ Data is **stationary** (no trend, no seasonality)
- ✅ Short-term forecasting (a few steps ahead)
- ✅ Need simple, interpretable model
- ✅ Computational efficiency is important

### When NOT to Use SES:
- ❌ Data has clear **trend** (use Holt's method instead)
- ❌ Data has **seasonality** (use Holt-Winters instead)
- ❌ Need long-term forecasts (forecasts flatten to constant)
- ❌ Multiple predictors available (use regression models)

### Alpha Parameter Guidelines:
- **α → 1**: Responsive to recent changes, less smoothing (volatile series)
- **α → 0**: More smoothing, less responsive (stable series)
- **Typical range**: 0.1 to 0.3 for most applications
- **Optimization**: Let statsmodels find optimal α via MLE

### Limitations:
1. **Flat forecasts**: All future forecasts are the same value
2. **No uncertainty**: Base model doesn't provide prediction intervals
3. **Stationarity required**: Must difference or transform non-stationary data first
4. **Short horizon**: Best for 1-10 steps ahead